In [ ]:
from pathml.core import HESlide, Tile, types
from pathml.preprocessing import BaseTransform
import matplotlib.pyplot as plt
from pathml.preprocessing import Pipeline, TissueDetectionHE
from pathml.utils import plot_mask, RGB_to_GREY
from pathml.preprocessing import (
    BoxBlur, GaussianBlur, MedianBlur,
    NucleusDetectionHE, StainNormalizationHE, SuperpixelInterpolation,
    ForegroundDetection, TissueDetectionHE, BinaryThreshold,
    MorphClose, MorphOpen
)

class ConvertToRGB(BaseTransform):
    def apply(self, tile):
        if len(tile.image.shape) == 2 or (tile.image.shape[2] == 1):
            # Convert grayscale to RGB
            tile.image = cv2.cvtColor(tile.image, cv2.COLOR_GRAY2RGB)

            
fontsize = 14

wsi = HESlide("/home/pathml/wsi/normal4.tif")

print(wsi)

region = wsi.slide.extract_region(location = (0, 0), size = (15360, 12800))

# reshape the region from 5dim to 3 / got all kinds of errors like TypeError: Invalid shape (3685, 1251, 1, 3, 1) for image data, also openCV complains about not finding the chanels
region = region[:, :, 0, :, 0]

# https://pathml.readthedocs.io/en/latest/api_core_reference.html#tile
# region needs to be of np.ndarray type
# what converts the image to np.ndarray type is the extract_region function:
# https://pathml.readthedocs.io/en/latest/api_core_reference.html#pathml.core.SlideData.extract_region

bigtile = Tile(region, coords = (0, 0), name = "testregion", slide_type = types.HE)

# Define a preprocessing pipeline
pipeline = Pipeline([
    ConvertToRGB(),
    TissueDetectionHE(mask_name="tissue", outer_contours_only=True),
])

# Run the preprocessing pipeline on each slide in the dataset
wsi.run(pipeline)